In [1]:
import sys
sys.path.append('/home/fedor/models')
sys.path.append('/home/fedor/models/slim')

In [2]:
import os

import functools
import tensorflow as tf
from google.protobuf import text_format

from object_detection import trainer
from object_detection import evaluator
from object_detection.builders import model_builder
from object_detection.builders import input_reader_builder
from object_detection.protos import pipeline_pb2
from object_detection.utils import label_map_util

In [3]:
config_path = 'ssd_mobilenet_v1_nexar.config'
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.gfile.GFile(config_path, 'r') as f:
    text_format.Merge(f.read(), pipeline_config)

model_config = pipeline_config.model
train_config = pipeline_config.train_config
input_config = pipeline_config.train_input_reader

In [4]:
model_fn = functools.partial(model_builder.build, model_config=model_config, is_training=True)
create_input_dict_fn = functools.partial(input_reader_builder.build, input_config)

In [5]:
!nvidia-smi

Wed Aug 30 19:11:39 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 378.13                 Driver Version: 378.13                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Graphics Device     Off  | 0000:01:00.0     Off |                  N/A |
| 27%   44C    P8    19W / 250W |   6940MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Graphics Device     Off  | 0000:02:00.0     Off |                  N/A |
| 24%   41C    P8    18W / 250W |      0MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
trainer.train(create_input_dict_fn, model_fn, train_config,
              master='', task=0, num_clones=1,
              worker_replicas=1, clone_on_cpu=False, ps_tasks=0,
              worker_job_name='lonely_worker', is_chief=True, train_dir='./logs/train')